# Environment

Run the appropriate "environment" code before running any solutions

In [ ]:
#@title environment: google.colab
fread = lambda path: from google.colab import files with open(path,'r') as f: f.read()

In [ ]:
#@title environment: vscode
fread = lambda path: open('inputs/'+path, 'r').read()

# Imports

Import libraries before running any solutions

In [172]:
import re, math
from operator import add

class formats:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    ORANGE = '\033[93m'
    RED = '\033[91m'
    WHITE = '\033[0m'
    BOLD = '\033[1m'

In [135]:
#@title Day 1: Trebuchet?!
ls = fread('23-1.txt').splitlines()

print(sum(map(lambda d: int(d[0]+d[-1]),[[c for c in l if '/' < c < ':'] for l in ls])))

ns = {n: n[0]+str(i)+n[-1] for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))}
[ls:= [l.replace(k,ns[k]) for l in ls] for k in ns]
print(sum([int(re.search('\\d',l).group()+re.search('\\d',l[::-1]).group()) for l in ls]))


54573
54591


In [136]:
#@title Day 2: Cube Conundrum
maxes = {'red': 12, 'green': 13, 'blue': 14}

ls = fread('23-2.txt').splitlines()
gs = [[{p.strip().split(' ')[1]: int(p.strip().split(' ')[0])
        for p in r.split(',')}
        for r in l.split(':')[1].split(';')]
        for l in ls]

valid = [i+1 for (i,g) in enumerate(gs)
             if all([all([maxes[k]>=r[k] for k in r]) for r in g])]
print(sum(valid))

powers = [math.prod([max([r.get(k,0) for r in g]) for k in maxes]) for g in gs]
print(sum(powers))

2727
56580


In [137]:
#@title Day 3: Gear Ratios
f = fread('23-3.txt').splitlines()

special = lambda x,y,n: [re.search('[^\\d.]',f[min(max(yi,0),len(f)-1)][max(x-1,0):min(x+n+1,len(f[0])-1)]) != None for yi in range(y-1,y+2)]
ps = {(m.start(),y): m.group() for (y,l) in enumerate(f) for m in re.finditer('\\d+',l)}
ns = [int(ps[(x,y)]) for (x,y) in ps if any(special(x,y,len(ps[(x,y)])))]
print(sum(ns))

stars = lambda x,y,n: [
  (m.start(),yi)
  for yi in range(max(0,y-1),min(y+2,len(f)))
  for m in re.finditer('\\*', f[yi]) if (x-2)<m.start()<(x+n+1)
]

ss = [(s, int(ps[(x,y)])) for (x,y) in ps for s in stars(x,y,len(ps[(x,y)]))]
pgs = [[n for t,n in ss if t==u] for u in set([s for s,n in ss])]
gs = [pg for pg in pgs if len(pg)==2]
print(sum([math.prod(g) for g in gs]))


529618
77509019


In [181]:
#@title Day 4-1: Scratchcards 
f = fread('23-4.txt').splitlines()

cs = [[set([int(n) for n in m.group().split(' ') if n != '']) 
  for m in re.finditer('[\\d ]+', l.split(':')[1])] 
  for l in f]

wins = [sum([1 for n in c[1] if n in c[0]]) for c in cs]
score = sum([1<<(w-1) for w in wins if w > 0])
print(score)

cards = [1]*len(wins)
[cards:= [sum(n) for n in zip(cards, 
  ([0]*(i+1))+([cards[i]]*w)+([0]*(len(wins)-w-i-1)))]
  for i,w in enumerate(wins)]
print(sum(cards))

18619
8063216


In [ ]:
#@title Day 5: If You Give A Seed A Fertilizer
f = [s.splitlines() for s in fread('23-5-0.txt').split('\n\n')]